In [1]:
from datetime import datetime
print(f'notebook last updated on {datetime.now().__str__()}')

notebook last updated on 2017-06-07 10:59:50.099461


# What does the phrase "Do not arise and do not be" mean in Isaiah 7:7?

> כה אמר אדני יהוה <br>
> לא תקום ולא תהיה <br>
> Isaiah 7:7

We will search for any other possible occurrences of this pairing.


## Search Strategy
Find clauses that have קום as the main verb followed by a daughter clause with היה as its main verb. We can add additional parameters such as the requirement that the daughter be in a coordinated relationship with its mother, as is the case in Isa 7:7.

In [2]:
# begin code, import the packages

import collections

from tf.fabric import Fabric

# instantiate TF
TF = Fabric(modules='hebrew/etcbc4c', silent=True)

# load our features
api = TF.load('''
              book chapter verse
              function pdp lex typ
              mother
              ''')

# globalize TF methods
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.06s B function             from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B pdp                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B lex                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.20s B typ                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.18s B mother               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s Feature overview: 103 for nodes; 5 for edges; 1 configs; 7 computed
  5.37s All features loaded/computed - for details use loadLog()


## Write a Function to Fulfill the Search

In [3]:
def qwm_hyh_match(clause_atom):
    '''
    Return match data as dict if a supplied clause
    atom node is a match for the requirements:
        * has qwm as its main verb
        * is followed by a daughter clause with hyh as its main
    '''
    
    # get the main verb
    verb = [word for phrase in L.d(clause_atom, otype='phrase')
               for word in L.d(phrase, otype='word')
               if F.function.v(phrase) in {'PreO', 'PreS', 'Pred'}
               and F.pdp.v(word) == 'verb'
           ]
    
    # no match if empty 
    # or raise error if multiples as sanity check
    if not verb:
        return None
    elif len(verb) > 1:
        raise Exception(f'More than one verb found in {clause_atom}...')
    else:
        verb = verb[0] # isolate verb
        verb_lex = F.lex.v(verb)  # get it's lexeme
    
    # check if verb is qwm
    if verb_lex != 'QWM[':
        return None
    
    # get daughters with matching parameters
    daughters = [ca for ca in E.mother.t(clause_atom)
                    
                    # Make sure clause atom is coordinate
                    # for clause atoms we do a workaround and look for the 'W' in typ
                    if F.typ.v(ca).startswith('W')
                 
                    # daughter should also be adjacent
                    and ca - clause_atom == 1
                ]
    
    # no match if no daughter, otherwise, isolate
    if not daughters:
        return None
    else:
        daughter = daughters[0]
    
    # get daughter main verb
    da_verb = [word for phrase in L.d(daughter, otype='phrase')
                   for word in L.d(phrase, otype='word')
                   if F.function.v(phrase) in {'PreO', 'PreS', 'Pred'}
                   and F.pdp.v(word) == 'verb'
               ]
    
    # no match if empty 
    # or raise error if multiples as sanity check
    if not da_verb:
        return None
    else:
        da_verb = da_verb[0] # isolate verb
        da_verb_lex = F.lex.v(da_verb)
    
    # check for HYH lexeme in daughter verb
    if da_verb_lex != 'HJH[':
        return None
    
    # everything up to this point is a match!
    return {'clause_atom': clause_atom,
            'daughter': daughter,
            'ca_verb':verb,
            'da_verb':da_verb
           }

# Apply the Function to all Clauses in HB

In [4]:
results = []

# loop through all HB clauses
for clause_atom in F.otype.s('clause_atom'):
    
    # apply the function
    check_match = qwm_hyh_match(clause_atom)
    
    # check for match and keep if so
    if check_match:
        results.append(check_match)
    
    # otherwise, move on to next ca
    else:
        continue
        
# report on results
print(f'{len(results)} results found!')

7 results found!


## Present Results

In [5]:
for result in results:
    
    passage = T.sectionFromNode(result['clause_atom'])
    ca_words = L.d(result['clause_atom'], otype='word')
    da_words = L.d(result['daughter'], otype='word')
    ca_text = T.text(ca_words)
    da_text = T.text(da_words)
    
    print(passage)
    print(ca_text)
    print(da_text)
    print()

('Exodus', 12, 30)
וַיָּ֨קָם פַּרְעֹ֜ה לַ֗יְלָה ה֤וּא וְכָל־עֲבָדָיו֙ וְכָל־מִצְרַ֔יִם 
וַתְּהִ֛י צְעָקָ֥ה גְדֹלָ֖ה בְּמִצְרָ֑יִם 

('Deuteronomy', 29, 12)
לְמַ֣עַן הָקִֽים־אֹתְךָ֩ הַיֹּ֨ום׀ לֹ֜ו לְעָ֗ם 
וְה֤וּא יִֽהְיֶה־לְּךָ֙ לֵֽאלֹהִ֔ים 

('Joshua', 4, 9)
וּשְׁתֵּ֧ים עֶשְׂרֵ֣ה אֲבָנִ֗ים הֵקִ֣ים יְהֹושֻׁעַ֮ בְּתֹ֣וךְ הַיַּרְדֵּן֒ תַּ֗חַת מַצַּב֙ רַגְלֵ֣י הַכֹּהֲנִ֔ים נֹשְׂאֵ֖י אֲרֹ֣ון הַבְּרִ֑ית 
וַיִּ֣הְיוּ שָׁ֔ם עַ֖ד הַיֹּ֥ום הַזֶּֽה׃ 

('Judges', 2, 18)
וְכִֽי־הֵקִ֨ים יְהוָ֥ה׀ לָהֶם֮ שֹֽׁפְטִים֒ 
וְהָיָ֤ה יְהוָה֙ עִם־הַשֹּׁפֵ֔ט 

('2_Samuel', 24, 11)
וַיָּ֥קָם דָּוִ֖ד בַּבֹּ֑קֶר פ 
וּדְבַר־יְהוָ֗ה הָיָה֙ אֶל־גָּ֣ד הַנָּבִ֔יא חֹזֵ֥ה דָוִ֖ד 

('Isaiah', 7, 7)
לֹ֥א תָק֖וּם 
וְלֹ֥א תִֽהְיֶֽה׃ 

('Ezekiel', 34, 29)
וַהֲקִמֹתִ֥י לָהֶ֛ם מַטָּ֖ע לְשֵׁ֑ם 
וְלֹֽא־יִהְי֨וּ עֹ֜וד אֲסֻפֵ֤י רָעָב֙ בָּאָ֔רֶץ 



## Reflection on Results

This query shows that with this particular construction, Isaiah 7:7 is unique. All other matches involve a different subject in the second, HYH clause, or they employ additional modifiers such as prepositions. 

There are some avenues to refine and expand this query:

* Restrict matches to cases wherein the two verbs agree in person, gender, and number. 
* Remove the restriction to QWM mother clauses to any predicated clause
* Look for cases where the daughter clause verb HYH appears without modifiers 